In [2]:
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

2023-08-10 14:47:18.888004: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import os 
from PIL import Image
import numpy as np

dirname_CLL, dirname_FL, dirname_MCL = 'lymphoma/CLL' ,'lymphoma/FL' ,'lymphoma/MCL'

final_CLL, final_FL , final_MCL  = [] , [] , []

for fname in os.listdir(dirname_CLL):
    im = Image.open(os.path.join(dirname_CLL, fname))
    imarray = np.array(im)
    final_CLL.append(imarray)

final_CLL = np.asarray(final_CLL) 

for fname in os.listdir(dirname_FL):
    im = Image.open(os.path.join(dirname_FL, fname))
    imarray = np.array(im)
    final_FL.append(imarray)

final_FL = np.asarray(final_FL) 


for fname in os.listdir(dirname_MCL):
    im = Image.open(os.path.join(dirname_MCL, fname))
    imarray = np.array(im)
    final_MCL.append(imarray)

final_MCL = np.asarray(final_MCL) 


In [4]:
print(final_CLL.shape)
print(final_FL.shape)
print(final_MCL.shape)

(113, 1040, 1388, 3)
(139, 1040, 1388, 3)
(122, 1040, 1388, 3)


In [5]:
final = np.concatenate((final_CLL, final_FL, final_MCL) , axis=0)
print(final.shape)

(374, 1040, 1388, 3)


In [6]:
label_CLL = np.zeros(final_CLL.shape[0])
label_FL = np.ones(final_FL.shape[0])
label_MCL = 2*np.ones(final_MCL.shape[0])

label = np.concatenate((label_CLL, label_FL, label_MCL) , axis=0)

# transform training label to one-hot encoding
lb = preprocessing.LabelBinarizer()
lb.fit(label)
label = lb.transform(label)

print(label.shape)


(374, 3)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(final, label, test_size=0.1, random_state=42)

In [8]:
img_size= (1040, 1388, 3)
n_classes = 3


In [9]:

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape = img_size, kernel_initializer = 'normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (5, 5), kernel_initializer = 'normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

model.summary

2023-08-10 14:47:26.921015: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f265a068ee0>>

In [ ]:
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    verbose=1
)

Epoch 1/10
